In [ ]:
from flask import Flask, render_template_string, request
from sklearn.preprocessing import MinMaxScaler
from flask import jsonify
import pandas as pd
from threading import Thread
import time
import pickle

app = Flask(__name__, static_url_path='/static')

# Load the pre-trained model
model = pickle.load(open('model.pkl','rb'))

# Load the scaler using pickle
with open('scaler.pkl', 'rb') as scaler_file:
    scaler = pickle.load(scaler_file)
    
html_template = """

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>ML Diabetes Prediction</title>
    <style>
          body {
            font-family: 'Arial', sans-serif;
            margin: 0;
            padding: 0;
            background-color: #ffe5ec; /* Light pale pastel pink color */
            display: flex;
            flex-direction: column;
            align-items: center;
        }

        .header {
            display: flex;
            align-items: center; /* Center vertically */
            justify-content: center; /* Center horizontally */
            background-color: #ffe5ec; /* Light pale pastel pink color */
            padding: 20px;
            text-align: center;
            height: 100vh;
        }
        .header img {
            max-width: 100%;
            height: auto;
            margin-right: 20px; /* Adjust as needed */
        }

        h1 {
            font-size: 60px; /* Adjust font size for the title */
            margin: 0; /* Remove default margin */
        }
        .text {
             text-align: left;
             max-width: 50%; /* Adjust as needed */
             color: #ff8fab;
        }
        h2 {
            text-align: center;
        }

        .container {
            display: flex;
            flex-direction: column;
            justify-content: center;
            align-items: center;
            padding: 20px;
        }


        label {
            font-size: 20px;
            margin-bottom: 8px;
            color: #333;
        }

        p {
            margin-top: 0;
            color: #666;
        }

        input {
            width: 100%;
            padding: 10px;
            box-sizing: border-box;
            border: 1px solid #ccc;
            border-radius: 4px;
            margin-top: 10px;
        }

        button {
            background-color: #4caf50; /* Green button background color */
            color: white;
            cursor: pointer;
            padding: 10px 16px;
            border: none;
            border-radius: 4px;
            margin-top: 10px;
        }

        button:hover {
            background-color: #45a049; /* Darker green on hover */
        }

        .box {
            background-color: #fff;
            padding: 30px; /* Remove or reduce this padding */
            border-radius: 8px;
            box-shadow: 0 0 15px rgba(0, 0, 0, 0.1);
            width: 600px;
            margin: 20px;
            display: flex;
            align-items: center;
        }

        .box .image-container {
            position: relative;
            width: 100px;
            height: 100px;
            margin: 0;
            padding: 40px;
            overflow: hidden; /* Ensure that any overflow is hidden */
        }

       .box img {
            position: absolute;
            top: 50%;
            left: 50%;
            transform: translate(-50%, -50%);
            max-width: 100%;
            max-height: 100%;
            width: auto;
            height: auto;
            margin: 0;
        }
        .box .text-container {
            padding-right: 30px;
            padding-left: 30px;
        }

        .input-container {
            flex: 1;
        }

        .input-container input {
            width: 100%;
            padding: 10px;
            box-sizing: border-box;
            border: 1px solid #ccc;
            border-radius: 4px;
            margin-top: 10px;
        }

        /* Adjusted margin for the input boxes to create space between them */
        .box:not(:last-child) {
            margin-bottom: 20px;
        }


    </style>
</head>

<body>
    <div class="header">
        <img src="{{ url_for('static', filename='diabetes.png') }}" alt="Diabetes Image">
        <div class="text">
            <h1>ML Diabetes Prediction</h1>
            <p>Diabetes is a chronic disease affecting millions worldwide. It occurs when the pancreas fails to 
            produce sufficient insulin or when the body struggles to effectively utilize the insulin it generates
            . Our predictive model uses logistic regression to assess an individual's risk of developing diabetes based on 
            six key inputs. Our model aims to contribute to the early 
            detection and management of diabetes, thereby mitigating its potentially devastating effects on
            health.</p>
            <div class="button-container">
                <a href="https://www.cdc.gov/diabetes/basics/index.html" target="_blank">
                    <button type="button">Learn More</button>
                </a>
            </div>
        </div>

    </div>
     <form method="post" action="/predict" class="container">
        <div class="box">
            <div class="image-container">
                <img src="{{ url_for('static', filename='gender.png') }}" alt="Gender Image">
            </div>
            <div class="text-container">
                <label for="input1">Gender</label>
                <p>Gender can be a factor in predicting diabetes.</p>
                <input type="text" id="input1" name="input1" placeholder="Enter Female or Male" required>
            </div>
        </div>

        <div class="box">
            <div class="image-container">
                <img src="{{ url_for('static', filename='hypertension.png') }}" alt="Gender Image">
            </div>
            <div class="text-container">
                <label for="input3">Hypertension</label>
                <p>Hypertension, also known as high blood pressure, occurs when pressure in your blood vessels is too high, typically 140/90 mmHg or above.</p>
                <input type="text" id="input3" name="input3" placeholder="Enter True or False" required>
            </div>
        </div>

        <div class="box">
            <div class="image-container">
                <img src="{{ url_for('static', filename='dia3.png') }}" alt="Gender Image">
            </div>
            <div class="text-container">
                <label for="input4">Heart Disease</label>
                <p>The term heart disease encompasses many different types of heart problems. </p>
                <input type="text" id="input4" name="input4" placeholder="Enter True or False" required>
            </div>
        </div>

        <div class="box">
            <div class="image-container">
                <img src="{{ url_for('static', filename='age.png') }}" alt="Gender Image">
            </div>
            <div class="text-container">
                <label for="input2">Age</label>
                <p>Age is an important factor in diabetes prediction. Older age is a risk factor for diabetes.</p>
                <input type="text" id="input2" name="input2" placeholder="Enter age" required>
            </div>
        </div>

        <div class="box">
            <div class="image-container">
                <img src="{{ url_for('static', filename='dia4.png') }}" alt="Gender Image">
            </div>
            <div class="text-container">
                <label for="input5">BMI</label>
                <p>Body mass index is calculated from the mass and height of a person. An acceptable range is 18.5 to 24.9.</p>
                <input type="text" id="input5" name="input5" placeholder="Enter BMI" required>
            </div>
        </div>

        <div class="box">
            <div class="image-container">
                <img src="{{ url_for('static', filename='blood.png') }}" alt="Gender Image">
            </div>
            <div class="text-container">
                <label for="input6">HbA1c Level</label>
                <p>HbA1c measures the amount of sugar attached to one’s red blood cells. A healthy range is <5.7% .</p>
                <input type="text" id="input6" name="input6" placeholder="Enter HbA1c level" required>
            </div>
        </div>

        <div class="box">
            <div class="image-container">
                <img src="{{ url_for('static', filename='dia5.png') }}" alt="Gender Image">
            </div>
            <div class="text-container">
                <label for="input7">Blood Glucose Level</label>
                <p>Blood glucose level measures the amount fo sugar in one’s blood. A good range is 70 to 100 mg/dL.</p>
                <input type="text" id="input7" name="input7" placeholder="Enter blood glucose level" required>
            </div>
        </div>

        <button type="submit">Predict</button>
    </form>
    {% if prediction is defined %}
        <p>Prediction: {{ prediction }}</p>
    {% endif %}
</div>
</body>

</html>

"""

@app.route('/', methods=['GET', 'POST'])
def index():
    return render_template_string(html_template)

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        try:
            # dropdown selects
            # get input values from the form and convert to what Log_Reg_Model sees
            if str(request.form['input1']) == "Male":
                input1 = 1 
            else:
                input1 = 0

            if str(request.form['input3']) == "True":
                input3 = 1 
            else:
                input3 = 0

            if str(request.form['input4']) == "True":
                input4 = 1 
            else:
                input4 = 0
            # input3 = str(request.form['input3'])
            # input4 = str(request.form['input4'])

            # text
            input2 = float(request.form['input2'])
            input5 = float(request.form['input5'])
            input6 = float(request.form['input6'])
            input7 = float(request.form['input7'])
            
            # Create a DataFrame with the input values
            input_data = pd.DataFrame({
                'age': [input2],
                'bmi': [input5],
                'HbA1c_level': [input6],
                'blood_glucose_level': [input7],
                'hypertension_1': [input3],
                'heart_disease_1': [input4],
                'gender_Male': [input1],
            })

            # Print the modified DataFrame
            print("Input Data:")
            print(input_data)

            # Rescale features using the loaded Min-Max scaler
            new_data_rescaled = scaler.transform(input_data)
            new_data_rescaled_df = pd.DataFrame(data=new_data_rescaled, columns=input_data.columns)

            print("Scaled Input Data:")
            print(new_data_rescaled)

            # Make a prediction using the pre-trained model
            prediction = model.predict(new_data_rescaled_df)
            print("Prediction:")
            print(prediction)


            # Display the prediction
            print("Prediction for the new input:", prediction[0])
            # Display the prediction on the web page
            return render_template_string(html_template, prediction=prediction[0])

        except ValueError as e:
            # Handle conversion errors
            return jsonify({"error": str(e)})

if __name__ == '__main__':

    flask_thread = Thread(target=app.run)
    flask_thread.start()

    # Allow some time for the Flask app to start
    time.sleep(2)

    # Additional code for other tasks if needed

    # Join the Flask thread to the main thread
    flask_thread.join()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [28/Nov/2023 19:51:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2023 19:51:24] "GET /static/diabetes.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2023 19:51:24] "GET /static/dia4.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2023 19:51:24] "GET /static/blood.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2023 19:51:24] "GET /static/dia3.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2023 19:51:24] "GET /static/hypertension.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2023 19:51:24] "GET /static/gender.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2023 19:51:24] "GET /static/dia5.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2023 19:51:24] "GET /static/age.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2023 19:52:25] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2023 19:52:25] "GET /static/dia4.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2023 19:52:25] "GET /static/diabetes.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2023 19:52:25] "GET /stat

Input Data:
    age   bmi  HbA1c_level  blood_glucose_level  hypertension_1  \
0  25.0  20.0          4.0                 80.0               0   

   heart_disease_1  gender_Male  
0                0            0  
Scaled Input Data:
[[0.31181181 0.11659664 0.09090909 0.         0.         0.
  0.        ]]
Prediction:
[False]
Prediction for the new input: False


127.0.0.1 - - [28/Nov/2023 19:52:55] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2023 19:52:55] "GET /static/diabetes.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2023 19:52:55] "GET /static/gender.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2023 19:52:55] "GET /static/age.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2023 19:52:55] "GET /static/dia4.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2023 19:52:55] "GET /static/hypertension.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2023 19:52:55] "GET /static/blood.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2023 19:52:55] "GET /static/dia3.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2023 19:52:55] "GET /static/dia5.png HTTP/1.1" 304 -


Input Data:
    age   bmi  HbA1c_level  blood_glucose_level  hypertension_1  \
0  86.0  27.0          7.0                120.0               1   

   heart_disease_1  gender_Male  
0                1            1  
Scaled Input Data:
[[1.07507508 0.19829599 0.63636364 0.18181818 1.         1.
  1.        ]]
Prediction:
[ True]
Prediction for the new input: True
